<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/HF_LOCAL_download-Mixtral-8x7B-Instruct-v0.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://huggingface.co/docs/hub/en/models-downloading

https://huggingface.co/docs/huggingface_hub/en/guides/download

In [ ]:
!pip install bitsandbytes -q
!pip install accelerate -q
!pip install sentencepiece -q
!pip install colab-env --quiet

In [2]:
!nvidia-smi

Thu May 23 23:00:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              45W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive',readonly=False,force_remount=True)

In [ ]:
%mkdir -p /content/gdrive/MyDrive/model

In [3]:
%cd /content/gdrive/MyDrive/model

/content/gdrive/MyDrive/model


In [4]:
!git lfs install

Git LFS initialized.


In [ ]:
#DONE
#!git clone https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1

In [4]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
## firts
#!git clone https://huggingface.co/abacusai/Smaug-72B-v0.1"

### second
%cd /content/gdrive/MyDrive/model/
!git lfs install
!git clone https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1

#152334H/miqu-1-70b-sf

In [ ]:
#!huggingface-cli download gpt2 config.json
%cd /content/gdrive/MyDrive/model/
!huggingface-cli download gpt2 config.json model.safetensors

In [ ]:
from huggingface_hub import snapshot_download
snapshot_download()

nvidia/Llama3-ChatQA-1.5-70B

https://huggingface.co/nvidia/Llama3-ChatQA-1.5-70B

In [ ]:
from huggingface_hub import snapshot_download
snapshot_download(repo_id="nvidia/Llama3-ChatQA-1.5-70B",local_dir="/content/gdrive/MyDrive/model/Llama3-ChatQA-1.5-70B")


abacusai/Smaug-72B-v0.1

https://huggingface.co/abacusai/Smaug-72B-v0.1

In [ ]:
from huggingface_hub import snapshot_download
snapshot_download(repo_id="abacusai/Smaug-72B-v0.1",local_dir="/content/gdrive/MyDrive/model/Smaug-72B-v0.1")

mistralai/Mixtral-8x7B-Instruct-v0.1

In [ ]:
from huggingface_hub import snapshot_download
snapshot_download(repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",local_dir="/content/gdrive/MyDrive/model/")

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
#from transformers import FalconForCausalLM
from transformers import pipeline

In [7]:
# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

from transformers import AutoConfig

In [9]:
from transformers import AutoModelForCausalLM


model_id = '/content/gdrive/MyDrive/model/' ## mistralai/Mixtral-8x7B-Instruct-v0.1

#model_id = '/content/gdrive/MyDrive/model/Smaug-72B-v0.1' ## NEED more than 40GB GPU

#model_id = '/content/gdrive/MyDrive/model/Llama3-ChatQA-1.5-70B'

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    #attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)


Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [17]:
prompt="What was the first album Beyoncé released as a solo artist?"
#prompt="What is the capital of Russia?"

prompt="Can you explain the concepts of Quantum Computing?"
prompt = f"Instruct: Answer the following question.\n{prompt}"

#prompt = f"Instruct: Answer the following question.\n{prompt}\nOutput:\n" # for dataset_squad2

In [18]:
def get_answer(prompt):
  outputs = pipe(prompt, max_new_tokens=512, temperature=0.8, do_sample=True, top_k=50, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.eos_token_id)
  return outputs

In [19]:
outputs = get_answer(prompt)

Knowledge Case #1

In [20]:
print('Question: %s'%prompt)
print()
#print('Answer: %s \nOutput:\n'%outputs[0]['generated_text'])
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")
#del outputs

Question: Instruct: Answer the following question.
Can you explain the concepts of Quantum Computing?

Generated Answer:
Answer: Quantum computing is a type of computation that uses quantum mechanics to perform operations on data. It differs from traditional computing, which uses bits to represent information and performs operations on those bits using classical logic gates.

In quantum computing, data is represented using quantum bits, or qubits, which can exist in a superposition of states. This means that a qubit can be in a state of 0, 1, or both at the same time. This allows quantum computers to perform certain operations much faster than classical computers.

One of the key concepts in quantum computing is the idea of quantum entanglement. When two qubits are entangled, their states are correlated in such a way that the state of one qubit cannot be described independently of the state of the other. This allows quantum computers to perform certain operations in parallel, which can

FIRST REASONING CASE

In [34]:
%%time
prompt = "I bought a computer for $900, sold it for $1200, repurchased it for $1300, and sold it again for $1600. how much did I earn? Take into consideration the money for the repurchased, too."
outputs = get_answer(prompt)

CPU times: user 31.9 s, sys: 46.4 ms, total: 31.9 s
Wall time: 31.8 s


In [35]:
print('Question: %s'%prompt)
print()
#print('Answer: %s \nOutput:\n'%outputs[0]['generated_text'])
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")
del outputs

Question: I bought a computer for $900, sold it for $1200, repurchased it for $1300, and sold it again for $1600. how much did I earn? Take into consideration the money for the repurchased, too.

Generated Answer:
Ok so here's what you should do:

1. First calculate the profit from the first sale:
$1200-$900=$300

2. Next calculate the loss from the repurchase:
$1300-$1200=$100

3. Then calculate the profit from the second sale:
$1600-$1300=$300

4. Finally add the profit from the first sale and the second sale and subtract the loss from the repurchase:
$300+$300-$100= $500

So you earned $500.


SECOND REASONING CASE

In [26]:
%%time
prompt = "I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering."
outputs = get_answer(prompt)

CPU times: user 15.7 s, sys: 27.2 ms, total: 15.7 s
Wall time: 15.7 s


In [27]:
print('Question: %s'%prompt)
print()
#print('Answer: %s \nOutput:\n'%outputs[0]['generated_text'])
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Question: I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.

Generated Answer:
There were 6 kids so 6 cones. 6 cones times $1.25 per cone equals $7.50. If I gave the man $10, then he would give me back $2.50. I got back around $2.50.

Answer: $2.50


Knowledge Case #2

In [24]:
prompt="What was the first album Beyoncé released as a solo artist?"
#prompt="What is the capital of Russia?"
outputs = get_answer(prompt)


In [25]:
print('Question: %s'%prompt)
print()
#print('Answer: %s \nOutput:\n'%outputs[0]['generated_text'])
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Question: What was the first album Beyoncé released as a solo artist?

Generated Answer:
Beyoncé released her first solo album, Dangerously in Love, on 23 June 2003. The album debuted at number one on the Billboard 200 chart and sold 317,000 copies in its first week. It was certified 4x Platinum by the Recording Industry Association of America (RIAA) on 11 January 2004. The album spawned the hit singles "Crazy in Love", "Baby Boy", "Me, Myself and I", and "Naughty Girl", which all topped the Billboard Hot 100 chart.
